In [30]:
from pypdf import PdfReader 
import glob
import os
import pandas as pd
import re
import numpy as np 

In [31]:
def find_int(pattern, text):
    match = re.search(pattern, text)
    if match:
        val = int(match.group(1))
        return val
    else:
        return np.nan

In [32]:
def find_str(pattern, text):
    match = re.search(pattern, text)
    if match:
        val = match.group(1)
        return val
    else:
        return None

In [33]:
cwd = os.getcwd()
directory = cwd + '/datasets/old_data/hastaverileri/'
pdf_files = glob.glob(os.path.join(directory, '**', '*.pdf'), recursive=True)

In [36]:
columns = ['patient' ,'patient_name', 'application_date', 'patient_no', 
           'report_date', 'age', 'sex',
           'practicing_doctor', 'requesting_section', 'preliminary_diagnosis',
           'aortic_root', 'ascending_aorta', 'end_diastolic_diameter',
           'end_systolic_diameter', 'injection_fraction', 'septum',
           'back_wall', 'hatchway_speed', 'local_wall',
           'left_atrium_size', 'right_atrium_size', 'diastolic_diameter',
           's_val', 'tapse', 'visual_systolic_function', 
           'e_a', 'deceleration_time', 'ivrz', 'e_septal', 'e_lateral',
           'systolic_pulmonary_artery_pressure', 'result']

df = pd.DataFrame(columns=columns)

In [37]:
dataframes = []
for file in pdf_files:
    patient = int(re.search(r'/(\d+)\.pdf$', file).group(1))
    reader = PdfReader(file)   
    page = reader.pages[0] 
    pdf = page.extract_text()
    pdf = pdf.split("Hasta Bilgileri\n")[1]
    patient_name = find_str(r'Adı Soyadı:\s*([A-ZÇĞİÖŞÜa-zçğıöşü]+\s+[A-ZÇĞİÖŞÜa-zçğıöşü]+)', pdf)
    pattern = r'Uygulama Tarihi:\s*(\d{2}\.\d{2}\.\d{4}\s\d{2}:\d{2}:\d{2})'
    application_date = find_str(pattern, pdf)
    pattern = r'Hasta No: (\d+)'
    patient_no = find_int(pattern, pdf)
    pattern = r'Rapor Tarihi:\s*(\d{2}\.\d{2}\.\d{4}\s\d{2}:\d{2}:\d{2})'
    report_date = find_str(pattern, pdf)
    pattern = r'Hasta No: (\d+)'
    patient_no = find_int(pattern, pdf)
    spl = pdf.split("\n")
    split2 = spl[2].split(" Uygulayan Doktor:")
    age_sex = split2[0][14:].split(" / ")
    age = int(age_sex[0])
    sex = age_sex[1]
    practicing_doctor = split2[1]
    split3 = spl[3].split(" Ön Tanı:")
    requesting_section = split3[0][14:]
    preliminary_diagnosis = split3[1]
    pattern = r'Aort kökü \(cm\) (\d+)'
    aortic_root = find_int(pattern, pdf)
    pattern = r'Çıkan aorta \(cm\) (\d+)'
    ascending_aorta = find_int(pattern, pdf)
    pattern = r'Diyastol sonu çapı \(cm\) (\d+)'
    end_diastolic_diameter = find_int(pattern, pdf)
    pattern = r'Sistol sonu çapı \(cm\) (\d+)'
    end_systolic_diameter = find_int(pattern, pdf)
    pattern = r'Ejeksiyon Fraksiyonu \(%\) (\d+)'
    injection_fraction = find_int(pattern, pdf)
    pattern = r'Septum \(cm\) (\d+)'
    septum = find_int(pattern, pdf)
    pattern = r'Arka duvar \(cm\) (\d+)'
    back_wall = find_int(pattern, pdf)
    pattern = r'Çıkım yolu hızı (m/sn) (\d+)'
    hatchway_speed = find_int(pattern, pdf)
    pattern = r'Bölgesel duvar hareketleri : (\d+)'
    local_wall = find_int(pattern, pdf)
    pdf1 = pdf.split("*: Ön tanı veya bulgular doğrultusunda sol ventrikül sistol ve diyastol\nhacimleri ve atım hacmini eklemek gerekebilir.")[1]
    pdf1, pdf2 = pdf1.split("†: Ölçüm penceresi belirtilerek, *: Ön tanı veya bulgular\ndoğrultusunda sağ ventrikül çıkım yolu velositesi gerekebilir,\nekojenitesi iyi olan hastalarda fraksiyonel alan değişiminin\nhesaplanması tercih edilmelidir.")
    pattern = r'Sol atriyum boyutu\s*\(cm\)\s*(\d+)'
    left_atrium_size = find_int(pattern, pdf1)
    pattern = r'Sağ atriyumboyutu\s*\(cm\)\s*(\d+)'
    right_atrium_size = find_int(pattern, pdf1)
    pattern = r'Diyastol çapı\(cm\)† (\d+)'
    diastolic_diameter = find_int(pattern, pdf1)
    pattern = r'S \(cm/sn\) (\d+)'
    s_val = find_int(pattern, pdf)
    pattern = r'TAPSE \(cm\) (\d+)'
    tapse = find_int(pattern, pdf)
    pattern = r'Görsel sistolik işlev\s*normal\s*/\s*azalmış\s*(\d+)'
    visual_systolic_function = find_int(pattern, pdf1)
    sp2 = pdf2.split('\n')
    e_a = int(sp2[1][11:-8]) if sp2[1][11:-8] and sp2[1][11:-8].isdigit() else np.nan
    pattern = r'Deselerasyon süresi (ms) (\d+)'
    deceleration_time = find_int(pattern, pdf2)
    ivrz = int(sp2[3][11:-6]) if sp2[3][11:-6] and sp2[3][11:-6].isdigit() else np.nan
    e_septal = int(sp2[4][11:-4]) if sp2[4][11:-4] and sp2[4][11:-4].isdigit() else np.nan
    e_lateral = int(sp2[5][11:-4]) if sp2[5][11:-4] and sp2[5][11:-4].isdigit() else np.nan
    pattern = r'Sistolik pulmoner arter basıncı \(mmHg\):\s*(\d+)'
    systolic_pulmonary_artery_pressure = find_int(pattern, pdf2)
    split3 = pdf2.split("Sonuçlar")[1]
    result = split3.split("\nAsistan")[0]
    df.loc[len(df)] = [patient, patient_name, application_date, patient_no, report_date, age, 
                   sex, practicing_doctor, requesting_section, preliminary_diagnosis, 
                   aortic_root, ascending_aorta, end_diastolic_diameter, end_systolic_diameter, 
                   injection_fraction, septum, back_wall, hatchway_speed, local_wall, left_atrium_size, 
                   right_atrium_size, diastolic_diameter, s_val, tapse, visual_systolic_function, 
                   e_a, deceleration_time, ivrz, e_septal, e_lateral, systolic_pulmonary_artery_pressure, result]


In [38]:
df

,patient,patient_name,application_date,patient_no,report_date,age,sex,practicing_doctor,requesting_section,preliminary_diagnosis,...,s_val,tapse,visual_systolic_function,e_a,deceleration_time,ivrz,e_septal,e_lateral,systolic_pulmonary_artery_pressure,result
0,3,VEZİR YAŞAR,None,2022238881,18.10.2023 16:08:33,60,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA: 35X42X52
1,23,SARAYNAZ GÖKBAŞİ,None,2009334527,20.10.2023 15:05:57,76,K,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 41X45X55)\nHAFİF TY
2,25,HÜSNİYE OLAM,None,2009444601,20.10.2023 16:38:37,63,K,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 44X50X56)\nMİTRAL ANULUS K...
3,27,İLHAMİ YEŞİL,None,2009392364,23.10.2023 15:43:41,57,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 40X43X48)\nSOL VENTRİKÜL H...
4,22,HANİFE ÇİÇEK,None,2010223331,20.10.2023 14:57:16,65,K,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,"ASENDAN AORTADA DİLATASYON, LA DİLATASYONU (LA..."
5,19,MURAT KARAYEL,None,2013115566,20.10.2023 14:08:10,47,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 41X41X54)\nHAFİF MY
6,14,YUSUF TURHAN,None,2010268947,19.10.2023 15:40:13,66,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA: 38X44X45\nHAFİF TY
7,13,DURMUŞ ÖZTÜRK,None,2009166562,19.10.2023 15:32:36,51,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA: 39X48X52\nMİNİMAL TY
8,41,HACER BİNALİOĞLU,None,2010361726,27.10.2023 15:57:16,64,K,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 43X36X55)
9,31,İDRİS SERT,None,2009070964,25.10.2023 15:37:47,56,E,,,,...,NaN,NaN,0,NaN,NaN,NaN,50,NaN,NaN,LA DİLATASYONU (LA: 42X47X53)


,MergedColumn
0,LA: 35X42X52
1,LA DİLATASYONU (LA: 41X45X55)
2,LA DİLATASYONU (LA: 44X50X56)
3,LA DİLATASYONU (LA: 40X43X48)
4,"ASENDAN AORTADA DİLATASYON, LA DİLATASYONU (LA..."
...,...
111,HAFİF AY/MY
116,"HAFİF AY, ORTA MY"
120,HAFİF MY/TY
122,"ORTA MY, İLERİ TY"


In [39]:
df.isna().sum()

patient                                0
patient_name                           0
application_date                      41
patient_no                             0
report_date                            0
age                                    0
sex                                    0
practicing_doctor                      0
requesting_section                     0
preliminary_diagnosis                  0
aortic_root                            0
ascending_aorta                        0
end_diastolic_diameter                41
end_systolic_diameter                  0
injection_fraction                     0
septum                                 0
back_wall                              0
hatchway_speed                        41
local_wall                            41
left_atrium_size                       0
right_atrium_size                      0
diastolic_diameter                    40
s_val                                 41
tapse                                 41
visual_systolic_

In [40]:
df.dtypes

patient                                 int64
patient_name                           object
application_date                       object
patient_no                              int64
report_date                            object
age                                     int64
sex                                    object
practicing_doctor                      object
requesting_section                     object
preliminary_diagnosis                  object
aortic_root                             int64
ascending_aorta                         int64
end_diastolic_diameter                float64
end_systolic_diameter                   int64
injection_fraction                      int64
septum                                  int64
back_wall                               int64
hatchway_speed                        float64
local_wall                            float64
left_atrium_size                        int64
right_atrium_size                       int64
diastolic_diameter                

In [41]:
df['report_date'] = pd.to_datetime(df['report_date'], format='%d.%m.%Y %H:%M:%S')
pd.set_option('future.no_silent_downcasting', True)
df['practicing_doctor'] = df['practicing_doctor'].replace('', np.nan)
df['requesting_section'] = df['requesting_section'].replace('', np.nan)
df['preliminary_diagnosis'] = df['preliminary_diagnosis'].replace('', np.nan)

In [42]:
for col in df.columns:
    print(col)
    print(df[col].unique())
    print('--------------------------------')

patient
[ 3 23 25 27 22 19 14 13 41 31 29  8 39 40 10 17  2  1 34 21 12 38 37 26
  5  7 32 15 28 16 24 33 20  9 11 18 35  4 36  6 30]
--------------------------------
patient_name
['VEZİR YAŞAR' 'SARAYNAZ GÖKBAŞİ' 'HÜSNİYE OLAM' 'İLHAMİ YEŞİL'
 'HANİFE ÇİÇEK' 'MURAT KARAYEL' 'YUSUF TURHAN' 'DURMUŞ ÖZTÜRK'
 'HACER BİNALİOĞLU' 'İDRİS SERT' 'NÜSRET GÖKÇE' 'LÜTFİYE ADALI'
 'ERDOĞAN YILMAZ' 'AYŞE YILDIRIM' 'KADİR YALÇIN' 'ŞAHİDE NİYAZOĞLU'
 'MÜNÜR ÇAPKINLAR' 'NURCAN TANTUR' 'SÜHEYLA OKUMUŞ' 'AZİZE OPUZ'
 'EMİNE ARINÇ' 'GÜLSİMET TURHANER' 'NAZMİYE MÜMİNOĞLU' 'TÜLAY BAKIR'
 'OĞUZHAN GÜLOĞULLARI' 'SEVDA UYSAL' 'İZZET SOYLU' 'EROL ALTUNKAYA'
 'HACI ERDAL' 'LEYLA YAĞMUR' 'NEZAHAT YILDIRIM' 'HATUN DOKMUŞ' 'AHMET ÖRS'
 'SELAMİ KOŞUCU' 'HANIM YILMAZ' 'NERİMAN GÜLER' 'NURAY KABA'
 'MENŞURE ÇİNİCİ' 'YUSUF SAÇLI' 'SENFİNAZ ARAS' 'FERİDE GÖZALAN']
--------------------------------
application_date
[None]
--------------------------------
patient_no
[2022238881 2009334527 2009444601 2009392364 2010223331 

In [43]:
df.to_csv('datasets/report.csv')